<a href="https://colab.research.google.com/github/lorenzrossi/InfoRet_SemanticSimilarity/blob/main/InfoRet_Semantic_Similarity_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, classification_report, hamming_loss
import shutil 
import sys
import time, datetime, json
!pip install sentencepiece
!pip install opendatasets
import opendatasets as op

import os

from keras_preprocessing.sequence import pad_sequences

# NLTK
import nltk as nlp
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
nlp.download('stopwords')
nlp.download('popular')
stop_words = stopwords.words('english')

# PYTORCH
!pip install transformers~=2.11.0
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, AdamWeightDecay, get_linear_schedule_with_warmup
import random
import gc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Down

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-01-26 12:37:02--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-01-26 12:37:03--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove.6B.zip.2      100%[===================>] 822.24M  5.04MB/s    in 3m 39s  

2023-01-26 12:40:42 (3.76 MB/s) - ‘glove.6B.zip.2’ saved [862182613/862182613]

Archive:  glove.6B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfi

In [ ]:
op.download("https://www.kaggle.com/datasets/stanfordu/stanford-natural-language-inference-corpus")
url = "https://www.kaggle.com/datasets/stanfordu/stanford-natural-language-inference-corpus"
dataset_folder ='stanford-natural-language-inference-corpus'
preprocessed_folder = 'snli-preprocessed'
saved_model_location = 'saved_models'
# bcc30972b8b9f25c2bc6c0a46d8f4d62

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: lorenzrossi
Your Kaggle Key: ··········


100%|██████████| 44.4M/44.4M [00:00<00:00, 74.8MB/s]


In [ ]:
SPECIAL_TOKENS = {
    'CLS': ['[CLS]'],
    'SEP': ['[SEP]']
}

SNLI_DATASET_URL = {
    "train_folder": "stanford-natural-language-inference-corpus/snli_1.0_train.csv",
    "validation_folder": "stanford-natural-language-inference-corpus/snli_1.0_dev.csv",
    "test_folder": "stanford-natural-language-inference-corpus/snli_1.0_test.csv"
}

SNLI_FILE_NAMES = {
    "train": "snli_1.0_train.csv",
    "validation": "snli_1.0_dev.csv",
    "test": "snli_1.0_test.csv"
}

# PREPROCESSING

In [ ]:
# Hyperparameters for preprocessing, tokenization and word embedding

max_len_tokens = 512

dataset_labels = {"contradiction": 0,
"entailment": 1,
"neutral": 2}

DATASET_LABELS = {}
for k, v in dataset_labels.items():
    DATASET_LABELS[k] = int(v)

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
DATASET_LABELS

{'contradiction': 0, 'entailment': 1, 'neutral': 2}

In [ ]:
class SNLIDataset(object):
    """
    Class to handle datasets and
    preprocess them in order to pass them on
    to the model for training/testing
    """

    def __init__(
        self, tokenizer, dataset_folder, preprocessed_folder, batch_size,
        max_len_tokens, dataset_labels, download_dataset=False, use_padding=True
    ):

        self.dataset_folder = dataset_folder
        self.preprocessed_folder = preprocessed_folder
        self.batch_size = batch_size
        self.use_padding = use_padding
        self.tokenizer = tokenizer
        self.max_len_tokens = max_len_tokens

        self.dataset_labels = dataset_labels

        if download_dataset:
            self.download_dataset()

        train_data = os.path.join(self.dataset_folder, SNLI_FILE_NAMES["train"])
        validation_data = os.path.join(self.dataset_folder, SNLI_FILE_NAMES["validation"])
        test_data = os.path.join(self.dataset_folder, SNLI_FILE_NAMES["test"])

        self.train_raw = pd.read_csv(train_data, index_col=1)
        self.train_raw = self.train_raw.sample(frac=0.3)
        
        self.eval_raw = pd.read_csv(validation_data, index_col=1)
        #self.eval_raw = self.eval_raw.sample(frac=0.5)

        self.test_raw = pd.read_csv(test_data, index_col=1)
        #self.test_raw = self.test_raw.sample(frac=0.5)

    def download_dataset(self):

       print("Downloading SNLI Dataset in CSV format")
       
       if not os.path.exists(self.dataset_folder):
           os.mkdir(self.dataset_folder)
       
       download_location = self.dataset_folder
       
       train_data_url = SNLI_DATASET_URL["train"]
       validation_data_url = SNLI_DATASET_URL["validation"]
       test_data_url = SNLI_DATASET_URL["test"]
       
       if os.path.exists(os.path.join(download_location)):
           print("Test Data already present")
       else:
           dataset = op.download(url, out=download_location)
       
       print(f"Dataset downloaded successfully in {download_location}")
       return

    def preprocess_dataset_util(self, dataset_df):
    
        tokenizer = self.tokenizer
        labels_dict = self.dataset_labels
        
        def tokenize_sentence(tokenizer, input_sentence):
        
            return tokenizer.tokenize(input_sentence)
            
        sentence_A = dataset_df.sentence1.to_numpy()
        sentence_B = dataset_df.sentence2.to_numpy()
        labels = dataset_df.gold_label.to_numpy()            
                 
        sentence_A_tokens = []
        sentence_B_tokens = []
        processed_labels = []

        for i, j, k in zip(sentence_A, sentence_B, labels):
            try:
                if k == '-':
                    continue
                
                t1 = tokenize_sentence(tokenizer, i)
                t2 = tokenize_sentence(tokenizer, j)
                
                sentence_A_tokens.append(t1)
                sentence_B_tokens.append(t2)
                
                label = labels_dict[k]
                processed_labels.append(label)
            
            except Exception as e:
                
                print(e)
                print("Skipping over sentence pair!")
                
                continue
        
        sentence_tokens = []
        input_ids = []
        token_lengths = []
        
        CLS_TOKEN = SPECIAL_TOKENS['CLS']
        SEP_TOKEN = SPECIAL_TOKENS['SEP']
        
        for i, j in zip(sentence_A_tokens, sentence_B_tokens):
        
            sentence = CLS_TOKEN + i + SEP_TOKEN + j + SEP_TOKEN
            
            token_ids = tokenizer.convert_tokens_to_ids(sentence)     
            sentence_tokens.append(sentence)
            
            token_lengths.append(len(token_ids))      
            input_ids.append(token_ids)
        
        return np.array(sentence_tokens), np.array(input_ids), np.array(token_lengths), np.array(processed_labels)
    
    def preprocess_dataset(self, d_partition="train_raw"):
    
        print(f"Preprocessing {d_partition} data")
        
        if d_partition.lower() not in ["train", "validation", "test"]:
            
            raise BaseException(
                "d_partition must be train, validation or test")
        
        if not os.path.exists(self.preprocessed_folder):
            os.mkdir(self.preprocessed_folder)
        
        preprocessed_location = self.preprocessed_folder
        file_name_base = os.path.join(preprocessed_folder, d_partition + "_")

        
        if os.path.exists(file_name_base + "tokens.npy"):           
            print("Retrieving tokens from .npy files")
            
            tokens = np.load(file_name_base + "tokens.npy", allow_pickle=True)
            ids = np.load(file_name_base + "token-ids.npy", allow_pickle=True)
            
            lengths = np.load(file_name_base + "token-lengths.npy", allow_pickle=True)
            labels = np.load(file_name_base + "labels.npy", allow_pickle=True)
        
        else:
            if d_partition.lower() == "train":
                dataset_df = self.train_raw        
            elif d_partition.lower() == "validation":
                dataset_df = self.eval_raw
            else:
                dataset_df = self.test_raw
            tokens, ids, lengths, labels = self.preprocess_dataset_util(dataset_df)
        
        print(f"Saving preprocessed {d_partition} data")
        
        return (tokens, ids, lengths, labels)
    
    def pad_and_create_attention_masks(self, input_ids):      
        max_len_tokens = self.max_len_tokens       
        if self.use_padding:
            input_ids = pad_sequences(input_ids, maxlen=max_len_tokens, dtype="long", value=0, truncating="post", padding="post")
        
        attention_masks = []       
        for sentence in input_ids:
            attention_mask = [int(token_id > 0) for token_id in sentence]
            attention_masks.append(attention_mask)
        
        return input_ids, attention_masks
    
    def convert_data_to_tensor_dataset(self, tokens, attention_masks, labels):
        
        batch_size = self.batch_size 

        tokens = torch.tensor(tokens)       
        attention_masks = torch.tensor(attention_masks)       
        labels = torch.tensor(labels)   

        data = TensorDataset(tokens, attention_masks, labels)
        sampler = RandomSampler(data)        
        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
        
        return (data, sampler, dataloader)
    
    def labels(self):
        return self.dataset_labels

In [ ]:
dataset = SNLIDataset(tokenizer = tokenizer, 
                      dataset_folder = dataset_folder, 
                      preprocessed_folder = preprocessed_folder, 
                      batch_size = 32, 
                      max_len_tokens = 256, 
                      dataset_labels = DATASET_LABELS, 
                      download_dataset=False, 
                      use_padding=True)

# Tokenization

In [ ]:
train_tokens, train_input_ids, train_token_lenghts, train_labels = dataset.preprocess_dataset(d_partition="train")

validation_tokens, validation_input_ids, validation_token_lenghts, validation_labels = dataset.preprocess_dataset(d_partition="validation")

test_tokens, test_input_ids, test_token_lenghts, test_labels = dataset.preprocess_dataset(d_partition="test")

print("Padding inputs and creating attention masks")

Preprocessing train data
'float' object has no attribute 'strip'
Skipping over sentence pair!


<ipython-input-14-061e3e295187>:114: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(sentence_tokens), np.array(input_ids), np.array(token_lengths), np.array(processed_labels)


Saving preprocessed train data
Preprocessing validation data
Saving preprocessed validation data
Preprocessing test data
Saving preprocessed test data
Padding inputs and creating attention masks


# Word embedding

In [ ]:
train_input_ids, train_attention_masks = dataset.pad_and_create_attention_masks(train_input_ids)

validation_input_ids, validation_attention_masks = dataset.pad_and_create_attention_masks(validation_input_ids)

test_input_ids, test_attention_masks = dataset.pad_and_create_attention_masks(test_input_ids)

print("Converting dataset to PyTorch TensorDataset")

Converting dataset to PyTorch TensorDataset


In [ ]:
train_data, train_sampler, train_dataloader = dataset.convert_data_to_tensor_dataset(train_input_ids, train_attention_masks, train_labels)

validation_data, validation_sampler, validation_dataloader = dataset.convert_data_to_tensor_dataset(validation_input_ids, validation_attention_masks, validation_labels)

test_data, test_sampler, test_dataloader = dataset.convert_data_to_tensor_dataset(test_input_ids, test_attention_masks, test_labels)

# BERT

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-05, eps=1e-08, weight_decay=0.0, correct_bias=True)

In [ ]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, batch in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = batch[0].to(device)
      mask_ids = batch[1].to(device)
      labels = batch[2].to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=None, 
                             attention_mask=mask_ids, 
                             labels=labels)

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, batch in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = batch[0].to(device)
        mask_ids = batch[1].to(device)
        labels = batch[2].to(device)
        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=None, 
                             attention_mask=mask_ids, 
                             labels=labels)
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
train(model, train_dataloader, validation_dataloader, optimizer)

Epoch 1: train_loss: 0.5998 train_acc: 0.7549 | val_loss: 0.5911 val_acc: 0.7615
00:28:04.52


KeyboardInterrupt: ignored